<a href="https://colab.research.google.com/github/Kiran-01/ImageGeneration-with-StyleGAN2/blob/main/StyleGAN_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 1.x
import tensorflow
print(tensorflow.__version__)

TensorFlow 1.x selected.
1.15.2


In [2]:
!git clone https://github.com/NVlabs/stylegan2.git

Cloning into 'stylegan2'...
remote: Enumerating objects: 138, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 138 (delta 4), reused 10 (delta 4), pack-reused 127
Receiving objects: 100% (138/138), 599.47 KiB | 5.76 MiB/s, done.
Resolving deltas: 100% (62/62), done.


In [3]:
!python /content/stylegan2/run_generator.py generate-images \
--network=gdrive:networks/stylegan2-ffhq-config-f.pkl \
 --seeds=4-6 --truncation-psi=0.4

Local submit - run_dir: results/00000-generate-images
dnnlib: Running run_generator.generate_images() on localhost...
Loading networks from "gdrive:networks/stylegan2-ffhq-config-f.pkl"...
Setting up TensorFlow plugin "fused_bias_act.cu": Preprocessing... Compiling... Loading... Done.
Setting up TensorFlow plugin "upfirdn_2d.cu": Preprocessing... Compiling... Loading... Done.
Generating image for seed 4 (0/3) ...
Generating image for seed 5 (1/3) ...
Generating image for seed 6 (2/3) ...
dnnlib: Finished run_generator.generate_images() in 1m 06s.


In [4]:
!ffmpeg -r 1 -i /content/results/00000-generate-images/seed%04d.png -vcodec mpeg4 -y people.mp4

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

In [5]:
!cp people.mp4 /content

cp: 'people.mp4' and '/content/people.mp4' are the same file


In [25]:
import sys
sys.path.insert(0, "/content/stylegan2")

In [26]:
import argparse
import numpy as np
import PIL.Image
import dnnlib
import dnnlib.tflib as tflib
import re
import sys
import pretrained_networks


In [27]:
def seeder(seeds, vector_size):
 result = []
 for seed in seeds:
   rnd = np.random.RandomState(seed)
   result.append( rnd.randn(1, vector_size))
 return result

In [28]:
def generate_images(gs, seeds, truncation_psi):
   noise_vars = [var for name, var in Gs.components.synthesis.vars.items() if name.startswith('noise')]
 
   Gs_kwargs = dnnlib.EasyDict()
   Gs_kwargs.output_transform = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)
   Gs_kwargs.randomize_noise = False
   if truncation_psi is not None:
       Gs_kwargs.truncation_psi = truncation_psi
 
   for seed_idx, seed in enumerate(seeds):
       print('Generating image for seed %d/%d ...' % (seed_idx, len(seeds)))
       rnd = np.random.RandomState()
       tflib.set_vars({var: rnd.randn(*var.shape.as_list()) for var in noise_vars}) # [height, width]
       images = Gs.run(seed, None, **Gs_kwargs) # [minibatch, height, width, channel]
       path = f"/content/out/image{seed_idx}.png"
       PIL.Image.fromarray(images[0], 'RGB').save(path)


In [29]:
network_pkl = "gdrive:networks/stylegan2-ffhq-config-f.pkl"
print('Loading networks from "%s"...' % network_pkl)
_G, _D, Gs = pretrained_networks.load_networks(network_pkl)

Loading networks from "gdrive:networks/stylegan2-ffhq-config-f.pkl"...
Setting up TensorFlow plugin "fused_bias_act.cu": Preprocessing... Loading... Done.
Setting up TensorFlow plugin "upfirdn_2d.cu": Preprocessing... Loading... Done.


In [30]:
vector_size = Gs.input_shape[1:][0]
seeds = seeder(range(6, 5), vector_size)

In [31]:
!mkdir /content/out

In [32]:
seeds = seeder([6, 5], vector_size)

In [33]:
STEPS = 300
diff = seeds[1] - seeds[0]
step = diff / STEPS
current = seeds[0].copy()
 
seeds2 = []
for i in range(STEPS):
 seeds2.append(current)
 current = current + step


In [34]:
generate_images(Gs, seeds2, truncation_psi=0.4)

Generating image for seed 0/300 ...
Generating image for seed 1/300 ...
Generating image for seed 2/300 ...
Generating image for seed 3/300 ...
Generating image for seed 4/300 ...
Generating image for seed 5/300 ...
Generating image for seed 6/300 ...
Generating image for seed 7/300 ...
Generating image for seed 8/300 ...
Generating image for seed 9/300 ...
Generating image for seed 10/300 ...
Generating image for seed 11/300 ...
Generating image for seed 12/300 ...
Generating image for seed 13/300 ...
Generating image for seed 14/300 ...
Generating image for seed 15/300 ...
Generating image for seed 16/300 ...
Generating image for seed 17/300 ...
Generating image for seed 18/300 ...
Generating image for seed 19/300 ...
Generating image for seed 20/300 ...
Generating image for seed 21/300 ...
Generating image for seed 22/300 ...
Generating image for seed 23/300 ...
Generating image for seed 24/300 ...
Generating image for seed 25/300 ...
Generating image for seed 26/300 ...
Generating 

In [35]:
!ffmpeg -r 50 -i /content/out/image%d.png -vcodec mpeg4 -y faces.mp4

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

In [36]:
!cp faces.mp4 /content

cp: 'faces.mp4' and '/content/faces.mp4' are the same file
